In [12]:
#lol y'a pas de module pour le torus on est dans la merde

In [13]:
# si k E Tp , il existe un entier r E [0;p] tq k = r/p
# on peut donc considérer k comme (r,p) ou juste comme r jsp

In [208]:
def AddInTore(p,a,b) :
    return (a+b)%p

In [209]:
def MultExtTore(p,n,x):
    if(n>=0):
        cpt = 0
        for i in range(n):
            cpt = AddInTore(p,cpt,x)
        return cpt
    return MultExtTore(p,-n,-x)

In [210]:
def TLWEncrypt(p,q,n,s,sigma,m):
    a=[]
    for i in range(n):
        a.append(randint(0,q-1))
    e = RealDistribution("gaussian",sigma).get_random_element()
    e = round(q*e)
    while(abs(e/q)>=1/(2*p)):
        e = RealDistribution("gaussian",sigma).get_random_element()
        e = round(q*e)
    #print("e= ",e)
    c = AddInTore(q,m*q/p,e)
    for i in range(n):
        c = AddInTore(q,c,MultExtTore(q,s[i],a[i]))
    return a+[c]

In [211]:
def TLWEDecrypt(p,q,n,s,c):
    a = c[:-1]
    m = c[-1]
    for i in range(n):
        m = AddInTore(q,m,MultExtTore(q,-s[i],a[i]))
    return round(m*p/q)%p

In [18]:
p = 16
q = 64
n = 5
s = [0,1,1,0,0]
sigma = 0.001

m = 10

c = TLWEncrypt(p, q, n, s, sigma, m)
print("c =",c)

mmaybe = TLWEDecrypt(p, q, n, s, c)
print("m =",mmaybe)

c = [35, 4, 57, 17, 18, 37]
m = 10


In [19]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt = 0
for i in range(10000):
    
    m = randint(0,p-1)


    c = TLWEncrypt(p, q, n, s, sigma, m)

    mmaybe = TLWEDecrypt(p, q, n, s, c)
    if(m==mmaybe):
        cpt = cpt +1
print(cpt)        

10000


In [212]:
def AddCiph(q,n,c1,c2):
    return [AddInTore(q,c1[i],c2[i]) for i in range(n+1)]

In [21]:
p = 2**9
q = 2**16
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(1000):
    
    m1 = randint(0,p-1)
    m2 = randint(0,p-1)

    c1,e1 = TLWEncrypt(p, q, n, s, sigma, m1)
    c2,e2 = TLWEncrypt(p, q, n, s, sigma, m2)

    m = TLWEDecrypt(p, q, n, s, AddCiph(q,n,c1,c2))
    if(m==AddInTore(p,m1,m2)):
        cpt1 = cpt1 +1
    elif(abs((e1+e2)/q)>=1/(2*p)):
        cpt2 = cpt2 + 1
print(cpt1)
print(cpt2)
print(cpt1+cpt2 ==1000)

ValueError: too many values to unpack (expected 2)

In [213]:
def MultCiph(q,n,K,c):
    return [MultExtTore(q,K,c[i]) for i in range(n+1)]

In [ ]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(1000):
    
    m = randint(0,p-1)

    c,e = TLWEncrypt(p, q, n, s, sigma, m)
    
    K = randint(1,100)
    mmaybe = TLWEDecrypt(p, q, n, s, MultCiph(q,n,K,c))
    
    if(mmaybe==MultExtTore(p,K,m)):
        cpt1 +=1
    elif(abs((K*e)/q)>=1/(2*p)):
        cpt2 +=1
print(cpt1)
print(cpt2)
print(cpt1+cpt2 == 1000)

In [214]:
def TrueGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = 1/(B^(j+1))
    return G
#G is the real gadget matrix

In [215]:
def UsedGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = q/(B^(j+1))
    return G
#G is the gadhet matrix of the numerator of the real gadget matrix with q for denominator

In [216]:
def AddInMatr(q,M1,M2):
    return [[AddInTore(q,M1[i][j],M2[i][j]) for j in range(len(M1[0]))] for i in range(len(M1))]

In [217]:
#Works with M a matrix with coeff in our Tq
def MultExtMatr(q,K,M):
    return [[MultExtTore(q,K,M[i][j]) for j in range(len(M[0]))] for i in range(len(M))] 

In [218]:
#M1 matrice entière et M2 matrice dans Tq
def MultInMatr(q,M1,M2) :
    M = [[0 for _ in range(len(M2[0]))] for _ in range(len(M1))]
    for i in range(len(M1)) :
        for j in range(len(M2[0])) :
            for k in range(len(M2)) :
                M[i][j] = AddInTore(q,M[i][j],MultExtTore(q,M1[i][k],M2[k][j]))
    return M

In [219]:
def TGSWEncrypt(q,B,l,n,m,s):
    sigma = 0.001
    Z = [TLWEncrypt(B**l,q,n,s,sigma,0) for i in range(n+1)]
    #print(Z)
    return AddInMatr(q,Z,MultExtMatr(q,m,UsedGadgetMatrix(q,B,l,n)))

In [220]:
def InvGVect(q,B,l,n,v):
    for i in range(n+1) :
        if v[i]> q/2 :
            v[i] = v[i]-q
    vbar = [round(B**l*v[i]/q) for i in range(n+1)]
    u=[]
    for i in range(n+1) :
        tmp = (vbar[i]).digits(B)
        while (len(tmp)!=l+1) :
            tmp.append(0)
        for j in range(len(tmp)-1) :
            if tmp[j]>= ceil(B/2) :
                tmp[j] = tmp[j]-B
                tmp[j+1] = tmp[j+1]+1
        tmp.reverse()
        u = u+(tmp[1:])
    return [u]

In [221]:
def InvGMatr(q,B,l,n,M) :
    return [InvGVect(q,B,l,n,M[i]) for i in range(len(M))]

In [222]:
g = InvG(64,4,3,1,[41,26])
G = UsedGadgetMatrix(64,4,3,1)
MultInMatr(64,g,G)

-23 [-3, -1, -1, 0]
26 [-2, -1, -2, 1]


[[41, 26]]

In [223]:
def MultInGSW(q,B,l,n,c1,c2) :
    return MultInMatr(q,InvGMatr(q,B,l,n,c1),c2)

In [224]:
#c1 est TGSW
#c2 est TLWE
def MultExtCiphLWE(q,B,l,n,c1,c2) :
    return MultInMatr(q,InvGVect(q,B,l,n,c2),c1)

In [250]:
k = 3
m = 5
q = 64
B = 4
l = 2
p = B**l
n = 500
sigma = 2**(-10)
cpt=0
for j in range(1000) :
    s = [randint(0,1) for i in range(n)]
    cm = TLWEncrypt(p,q,n,s,sigma,m)

    ck = TGSWEncrypt(q,B,l,n,m,s)
    c = MultExtCiphLWE(q,B,l,n,ck,cm)[0]
    if(TLWEDecrypt(p,q,n,s,c)==MultExtTore(p,k,m)) :
        cpt+=1
print(cpt)


KeyboardInterrupt: 